In [9]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Input, Conv2D,Flatten,Dense, BatchNormalization

In [49]:
class CNNBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = tf.keras.layers.Conv2D(
            filters,
            kernel_size,
            padding='same'
        )
        self.bn = tf.keras.layers.BatchNormalization()

    def call(self, input, training=False):
        x = self.conv(input)
        x = self.bn(x)
        x = tf.nn.relu(x)

        return x
    

class ResBlock(tf.keras.layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = tf.keras.layers.MaxPooling2D()
        self.identity_mapping = tf.keras.layers.Conv2D(
            channels[1], 3, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor),
                      training=training,)
        x = self.pooling(x)
        return x


# Specify input shape (assuming 150x150x3 images)
input_shape = (None,28, 28, 1)

# Build the model with specified input shape
# model = tf.keras.Sequential([
#     CNNBlock(32),
#     CNNBlock(64),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(10),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

model = tf.keras.Sequential([

    ResBlock([16,32,64]),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# Build the model with specified input shape

# Now, you can call model.summary()
model.build(input_shape= input_shape)
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_block_11 (ResBlock)     (None, 14, 14, 64)        24064     
                                                                 
 flatten_20 (Flatten)        (None, 12544)             0         
                                                                 
 dense_33 (Dense)            (None, 10)                125450    
                                                                 
 dense_34 (Dense)            (None, 1)                 11        
                                                                 
Total params: 149525 (584.08 KB)
Trainable params: 149301 (583.21 KB)
Non-trainable params: 224 (896.00 Byte)
_________________________________________________________________


In [40]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import os

# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# CNN -> BatchNorm -> ReLU (common structure)
# x10 (a lot of code to write!)


class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x


model = keras.Sequential(
    [CNNBlock(32), CNNBlock(64), CNNBlock(128),
     layers.Flatten(), layers.Dense(10),]
)


class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor),
                      training=training,)
        x = self.pooling(x)
        return x


class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

    def model(self):
        x = keras.Input(shape=(28, 28, 1))
        return keras.Model(inputs=[x], outputs=self.call(x))


model = ResNet_Like().model()
base_input = model.layers[0].input
base_output = model.layers[2].output
output = layers.Dense(10)(layers.Flatten()(base_output))
model = keras.Model(base_input, output)

In [36]:
# Now, you can call model.summary()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 res_block (ResBlock)        (None, 14, 14, 64)        28896     
                                                                 
 res_block_1 (ResBlock)      (None, 7, 7, 256)         592512    
                                                                 
 flatten_10 (Flatten)        (None, 12544)             0         
                                                                 
 dense_15 (Dense)            (None, 10)                125450    
                                                                 
Total params: 746858 (2.85 MB)
Trainable params: 745578 (2.84 MB)
Non-trainable params: 1280 (5.00 KB)
_________________________________________________________________
